<a href="https://colab.research.google.com/github/cchighman/ImageAI-YOLOv3-Fine-Tuning-Vehicle-Classification/blob/master/ImageAI_Fine_Tuning_Resnet50_for_Vehicle_Classification_based_on_YOLOV3_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip "cars34.zip"

In [103]:
!pip install imageai
!pip install tensorflow==1.15.0

In [0]:
import shutil
shutil.rmtree("cars65")

In [0]:
from imageai.Detection import ObjectDetection
import os
from PIL import Image
weights_path = "yolo.h5"

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( weights_path)
detector.loadModel()

In [0]:
!zip -r "cars_out.zip" "/content/cars"

In [0]:
image_path = "cars/test/prius/"
import copy
images = os.listdir(image_path)
for image in images:
    try:
        detections, objects_path = detector.detectObjectsFromImage(input_image=image_path+image, extract_detected_objects=True, output_image_path="/content/cars/test/prius/box_"+image, minimum_percentage_probability = 30)        
        for eachObject, eachObjectPath in zip(detections, objects_path):
            print(eachObject["name"] , " : " , str(eachObject["percentage_probability"]), " : ", str(eachObject["box_points"]))
            for detected in os.listdir(eachObjectPath):
                print("Copying " + eachObjecPath + "/" + detected + " to ../" + detected)
                copy.copy(eachObjectPath+"/"+detected, "../"+detected)

    except:
        pass

In [104]:
from imageai.Prediction.Custom import ModelTraining
model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("/content/cars")
model_trainer.trainModel(num_objects=2, num_experiments=100, save_full_model=True, transfer_with_full_training=True, transfer_from_model="resnet50_weights_tf_dim_ordering_tf_kernels.h5", enhance_data=True, batch_size=32, show_network_summary=True)

Training with all layers of the Model
Training using weights from a pre-trained model
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_477 (Conv2D)             (None, 112, 112, 64) 9472        input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization_477 (BatchN (None, 112, 112, 64) 256         conv2d_477[0][0]                 
__________________________________________________________________________________________________
activation_456 (Activation)     (None, 112, 112, 64) 0           batch_normalization_477[0][0]    
_____

In [0]:
!zip -r cars_trained cars/

In [0]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
pip install httplib2==0.15.0

In [0]:
!tar -cvzf model.tgz /content/cars/models/model_ex-093_acc-0.818203.h5

In [0]:

file5 = drive.CreateFile()
# Read file and set it as a content of this instance.
file5.SetContentFile('/content/resnet50_weights_tf_dim_ordering_tf_kernels.h5')

file5.Upload() # Upload the file.

In [0]:
from imageai.Prediction.Custom import CustomImagePrediction
prediction = CustomImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath( "/content/cars/models/model_ex-038_acc-0.994026.h5")
prediction.setJsonPath( "/content/cars/json/model_class.json")
prediction.loadModel(num_objects=191)

predictions, probabilities = prediction.predictImage("/content/cars/train/prius/00808_8eU6U08ZMn7_600x450.jpg", result_count=5)
for eachPrediction, eachProbability in zip(predictions, probabilities):
	print(eachPrediction, " : ", eachProbability)
